In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import blimpy as bl
%matplotlib inline

In [2]:
df = pd.read_csv('/home/cgchoza/galaxies/complete_cadences_catalog.csv')
df

,Target,Session,Band,Cadence ID,Frequency,.h5 path,.dat path,Time
0,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
1,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
2,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
3,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
4,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
...,...,...,...,...,...,...,...,...
36553,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
36554,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
36555,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
36556,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36


In [ ]:
chunk_size = 3000
group_size = 15
n_rows = len(df)

for batch_start in range(0, n_rows, group_size):
    batch_end = min(batch_start + group_size, n_rows)
    batch_arrays = []
    
    # Process rows batch_start … batch_end-1
    for idx in range(batch_start, batch_end):
        # load the waterfall
        fb = bl.Waterfall(df['.h5 path'].iloc[idx])
        data = np.squeeze(fb.data)
        n_chunks = data.shape[1] // chunk_size
        
        # slice into (n_chunks, freq, chunk_size)
        arr = np.array([data[:, j*chunk_size:(j+1)*chunk_size] for j in range(n_chunks)])
        batch_arrays.append(arr)
    
    # if every arr.shape == (n_chunks, freq, chunk_size), you can stack:
    batch_array = np.stack(batch_arrays, axis=0)
    # -> shape = (rows_in_batch, n_chunks, freq, chunk_size)
    
    out_path = f'/datax/scratch/jliang/data_array_rows_{batch_start}_{batch_end}.npy'
    np.save(out_path, batch_array)
    print(f"Saved rows {batch_start}–{batch_end-1} → {out_path}")
